In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import imageio
import os


# Define the objective function
def objective_function(x):
    return (6 * x - 2) ** 2 * np.sin(18 * x - 4)


# Define the UCB acquisition function with a higher `kappa` for exploration
def ucb_acquisition(X, gp, kappa=8.0):  # Increased kappa for more exploration
    mu, sigma = gp.predict(X, return_std=True)
    return mu + kappa * sigma


# Set up the initial plot grid and function
x = np.linspace(0, 1, 100).reshape(-1, 1)  # x values for plot grid
y = objective_function(x)  # true function values for the grid

# Initialize Gaussian Process with a flexible Matern kernel
kernel = Matern(length_scale=0.15, nu=2.5)  # Slightly adjusted length scale
gp = GaussianProcessRegressor(
    kernel=kernel, alpha=1e-5, normalize_y=True
)  # smaller alpha for closer fit

# Initial samples for better coverage
X_sample = np.array(
    [[0.05], [0.15], [0.35], [0.45], [0.75]]
)  # five evenly spaced initial points
y_sample = objective_function(X_sample)

# Storage for frames to create GIF
frames = []

# Optimization Loop
num_iterations = 15

for i in range(num_iterations):
    # Fit GP to current samples
    gp.fit(X_sample, y_sample)

    # Select next sample using UCB acquisition
    x_candidates = np.linspace(0, 1, 100).reshape(-1, 1)
    acquisition_values = ucb_acquisition(
        x_candidates, gp, kappa=5.0
    )  # Use updated kappa
    x_next = x_candidates[
        np.argmax(acquisition_values)
    ]  # Next point chosen by maximizing UCB

    # Sample the objective function at x_next
    y_next = objective_function(x_next.reshape(1, -1))

    # Add the new sample to the dataset
    X_sample = np.vstack((X_sample, x_next))
    y_sample = np.vstack((y_sample, y_next))

    # Predict using the GP for plotting
    mu, sigma = gp.predict(x, return_std=True)

    # Plot the current model state
    plt.figure(figsize=(12, 5))
    plt.plot(x, y, "r--", label="Objective Function")
    plt.plot(X_sample, y_sample, "bo", label="Samples")
    plt.fill_between(
        x.ravel(),
        mu - 1.96 * sigma,
        mu + 1.96 * sigma,
        alpha=0.2,
        color="blue",
        label="Confidence Interval",
    )
    plt.plot(x, mu, "b-", label="Surrogate Mean")
    plt.plot(x, acquisition_values, "g--", label="UCB Acquisition Function")
    plt.axvline(x=x_next, linestyle="--", color="k", label="Next Sample Point")
    plt.title(f"Iteration {i + 1}")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()

    # Save frame for GIF
    filename = f"frame_{i}.png"
    plt.savefig(filename)
    frames.append(filename)
    plt.close()

# Create GIF
with imageio.get_writer("bayesian_optimization.gif", mode="I", duration=1) as writer:
    for filename in frames:
        image = imageio.imread(filename)
        writer.append_data(image)

# # Remove individual frames after creating the GIF
# for filename in frames:
#     os.remove(filename)

/Users/ndas/miniforge3/envs/variationalinference/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/ndas/miniforge3/envs/variationalinference/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/ndas/miniforge3/envs/variationalinference/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.w